# Vorbereitung Machine Learning

Bevor du mit dem Machine Learning beginnen kannst, musst du einige Vorbereitungen treffen.

Einiges davon hast du schon erledigt, so sind z.B. die Dokumente bereits tokenisiert und die linguistische Analyse hast du auch durchgeführt. Dadurch kennst du die unterschiedlichen Wortarten.

Es bleiben aber noch einige Entscheidungen zu treffen.

## Daten einladen

Zuerst lädst du die Daten wie gewohnt ein. Du benutzt die Tabelle `nlp_articles`, um alle Freiheiten bzgl. der Daten (bzw. Tokens) zu haben.

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles", sql, index_col="id", parse_dates=["datePublished"])

## Anzahl der Dokumente

In [ ]:
len(df)

## Welche Entitäten analysieren?

Nun gibt es verschiedene Varianten, was du analysieren möchtest:

## Überschriften

Du könntest dich an erster Stelle für die Überschriften interessieren. Das ist besonders dann spannend, wenn du dich dafür interessierst, welche Überschriften angeklickt werden. In diesem Moment kennen die User ja den Text noch gar nicht - sie sehen nur die Überschriften. Deswegen wäre es dann genau richtig, nur diese zu analysieren (um z.B. vorherzusagen, welche primär angeklickt werden).

Schauen wir uns zunächst an, wie viele Überschriften es gibt:

In [ ]:
len(df[df["title"].map(str) != ''])

Wie erwartet gibt es keine Artikel, die keine oder leere Überschriften haben. 

Betrachten wir noch die grobe mittlere Anzahl der Tokens:

In [ ]:
df["title_length"] = [len(t.split(" ")) for t in df["title"]]
df["title_length"].describe()

Im Mittel sind das also etwa 8 Tokens, damit kann man schon einiges analysieren. Einer Analyse würde also nichts im Wege stehen.

## Überschriften und Einleitungen

Das gleich Verfahren können wir nun für die Überschriften mit den Einleitungen anwenden. Das wäre z.B. dann sinnvoll, wenn du durch ein *Eye Tracking* herausgefunden hast, dass die Leser meistens nur die Einleitung lesen, nicht aber den Rest der Meldung.

Gibt es Meldungen, die keine Einleitung haben?

In [ ]:
len(df[df["header"].map(str) == ''])

Zum Glück gibt es solche Meldungen auch nicht.

Berechnen wir noch die mittlere Anzahl der Tokens:

In [ ]:
df["title_header_length"] = [len(t.split(" ")) for t in (df["title"]+" "+df["header"])]
df["title_header_length"].describe()

Das sind im Mittel schon 27 Tokens. Mit dieser Datenmenge kannst du auch sehr gut arbeiten und damit Erkenntnisse ableiten.

## Gesamte Meldungen

Die gesamtem Meldungen sind bereits linguistisch analysiert. Die Anzahl der Tokens sind schon berechnet, die kannst du direkt auswerten:

In [ ]:
df["no_tokens"].describe()

Diese Texte sind *deutlich* länger und ihre Analyse wird daher wesentlich aufwändiger, aber bestimmt auch mehr Erkenntnisse hervorbringen. Eine solche Analyse ist aber nur dann sinnvoll, wenn du sichergehen kannst, dass die Leser auch die gesamte Meldung lesen werden. Sonst erzeugst du dadurch einen sog. *Bias*.

## Einzelne Absätze

Oft wirst du Texten begegnen, deren Absätze relativ unabhängig voneinander sind und unterschiedliche Themen behandeln. Bei diesen ist es dann sinnvoll, die Absätze separat voneinander zu analysieren.

Die Absätze sind mit `\n` voneinander getrennt und können so auch zerlegt werden:

In [ ]:
paragraphs = [p for text in df["text"] for p in text.split("\n")]

Wie viele Absätze gibt es insgesamt?

In [ ]:
len(paragraphs)

## Auswertungsentitäten richtig wählen

Bei der Wahl der richtigen Granularität für die Auswertung solltest du dir intensiv Gedanken machen. Überlege, was du erreichen willst. Du kannst das zwar später noch umstellen, aber eine richtige Wahl am Anfang spart erheblich Zeit und Ärger.